<a href="https://colab.research.google.com/github/carlos-alves-one/-BlockChain-Explorer-Project/blob/main/blockchain_explorer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Goldsmiths University of London
### MSc. Data Science and Artificial Intelligence
### Module: Blockchain Programming
### Author: Carlos Manuel De Oliveira Alves
### Student: cdeol003
### BlockChain Explorer Project

# Step 1: Setup
Explain the setup process, including importing necessary libraries (e.g., hashlib for cryptographic hashing) and any initial configurations required.

In [ ]:
import hashlib
